In [1]:
from robotic import ry
import numpy as np
import time

print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

ry version 0.0.19 compile time: Sep 23 2023 14:58:54
-- ry.cpp:operator():99(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000


In [2]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('push_cargo.g')
C.addFile('push_maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

In [3]:
C.view()

0

In [4]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])


ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.002842, evals: 54, done: 1, feasible: 0, sos: 29.5425, f: 0, ineq: 0, eq: 5.5 }
size of path: (1, 3)
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
----newton---- initial point f(x):292171 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):     266794  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):     242569  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):     219498  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     197700  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):     176934  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     157322  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):     138741  ACCEPT
--newton-- it:   8  |Delta|:        0.2  alpha:    

0

In [5]:
#define a path finding problem
rrt = ry.PathFinder()
i = 0
while True:
    i = i + 1
    print(i)
    rrt.setProblem(C, [qHome], [qT])
    ret = rrt.solve()
    path = ret.x
    print("rettttt: ", ret)
    #print("path size:", path)
    # display the path
    if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
        for t in range(0, path.shape[0] - 1):
            C.setJointState(path[t])
            time.sleep(0.002)
        break

1
rettttt:  { time: 0.33458, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
2
rettttt:  { time: 0.65915, evals: 10002, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
3
rettttt:  { time: 0.976903, evals: 15003, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
4
rettttt:  { time: 1.28976, evals: 20004, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
5
rettttt:  { time: 1.59865, evals: 24740, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [6]:
C.setJointState(path[0])
C.view()

0

In [7]:
bot = ry.BotOp(C, False)
bot.home(C)

    
for i in range(0, 600):
    i= i*100
    bot.moveTo(path[i])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

IndexError: index 700 is out of bounds for axis 0 with size 648